In [69]:
import pandas as pd
import numpy as np
import sys
import os
import bs4
from IPython.display import display_html,clear_output, HTML
import re
from datetime import datetime
import ast
import itertools
from tqdm import tqdm,trange
from file_tools import *
from request_tools import *
from parse_tools import *
from load_tools import *
import pyarrow as pa

pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 30)
pd.set_option('compute.use_numexpr', False)

import warnings
from pandas.errors import PerformanceWarning
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning
pd.options.mode.use_inf_as_na = True
warnings.filterwarnings('ignore', category=PerformanceWarning)
warnings.filterwarnings('ignore', category=DataConversionWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)

import pandas as pd
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif, RFE, mutual_info_regression
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from scipy.stats import pearsonr
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVR


In [70]:
SRC_DIR = './07-data-compiled/dataset-ss0110-mnmx-7fg'
FS_RANKED = pd.read_csv(f'{SRC_DIR}/features_scores_ranked/WL.csv',index_col=[0,1,2])
FS_RANKED_TOP_COLS = FS_RANKED.iloc[:150].index
list(FS_RANKED_TOP_COLS)

[('Opp_norm_minmax_tgl_basic_stats_cumu_avg', 'Result', 'W/L'),
 ('Team_norm_minmax_tgl_basic_stats_cumu_avg', 'Result', 'W/L'),
 ('Opp_norm_minmax_tgl_basic_stats_ven_cumu_avg', 'Result', 'W/L'),
 ('Team_norm_minmax_tgl_basic_stats_ven_cumu_avg', 'Result', 'W/L'),
 ('Team_norm_minmax_tgl_advanced_stats_cumu_avg', 'Advanced', 'TS%'),
 ('Opp_norm_minmax_tgl_advanced_stats_cumu_avg', 'Advanced', 'TS%'),
 ('Team_norm_minmax_tgl_advanced_stats_ven_cumu_avg', 'Advanced', 'ORtg'),
 ('Opp_norm_minmax_tgl_advanced_stats_ven_cumu_avg', 'Advanced', 'ORtg'),
 ('Opp_norm_minmax_tgl_advanced_stats_roll_08_avg',
  'Offensive Four Factors',
  'eFG%'),
 ('Team_norm_minmax_tgl_advanced_stats_roll_08_avg',
  'Offensive Four Factors',
  'eFG%'),
 ('Opp_norm_minmax_tgl_basic_stats_roll_08_avg', 'Result', 'W/L'),
 ('Opp_facts_venue_rest_days_tgl_basic', 'Match Info', 'H/A'),
 ('Team_norm_minmax_tgl_advanced_stats_ven_cumu_avg', 'Advanced', 'TS%'),
 ('Team_norm_minmax_tgl_basic_stats_roll_08_avg', 'Result',

In [71]:
FEATURES,LABELS = load_dataset(SRC_DIR)
FEATURES = FEATURES[FS_RANKED_TOP_COLS].dropna()
LABELS = LABELS.loc[FEATURES.index]['W/L']

FEATURES.shape,LABELS.shape

((19234, 150), (19234,))

In [72]:
# Drop all index with index < 20
FEATURES_RD_IDX = FEATURES.index.get_level_values(1)
FEATURES_AFTER_RD_15_75 = FEATURES.loc[(FEATURES_RD_IDX >= 15) & (FEATURES_RD_IDX <= 75)]
LABELS_AFTER_RD_15_75 = LABELS.loc[FEATURES_AFTER_RD_15_75.index]
FEATURES_AFTER_RD_15_75.sort_index(axis=1,level=[0,1],ascending=False)


Team_norm_minmax_tgl_basic_stats_ven_cumu_avg   
                                                                                           Team   
                                                                                            TRB   
                       index Team_id                                                              
/leagues/NBA_2023.html 15    /teams/ATL/2023.html                                      0.346535  \
                             /teams/BOS/2023.html                                      0.078261   
                             /teams/BRK/2023.html                                     -0.634783   
                             /teams/CHI/2023.html                                     -0.247525   
                             /teams/CHO/2023.html                                     -0.125000   
                             /teams/CLE/2023.html                                      0.075000   
                             /teams/DAL/2023.html                                     -0.786885   
                             /teams/DEN/2023.html                                     -0.305085   
                             /teams/DET/2023.html                                      0.216495   
                             /teams/GSW/2023.html                                     -0.413043   
                             /teams/HOU/2023.html                                     -0.016949   
                             /teams/IND/2023.html                                      0.089109   
                             /teams/LAC/2023.html                                     -0.934783   
                             /teams/LAL/2023.html                                      0.388889   
                             /teams/MEM/2023.html                                      0.775000   
...                                                                                         ...   
/leagues/NBA_2015.html 75    /teams/MIA/2015.html                                     -1.000000   
                             /teams/MIL/2015.html                                     -0.279461   
                             /teams/MIN/2015.html                                     -0.475676   
                             /teams/NOP/2015.html                                      0.066667   
                             /teams/NYK/2015.html                                     -0.656566   
                             /teams/OKC/2015.html                                      1.000000   
                             /teams/ORL/2015.html                                     -0.543860   
                             /teams/PHI/2015.html                                     -0.010526   
                             /teams/PHO/2015.html                                      0.269504   
                             /teams/POR/2015.html                                      0.713514   
                             /teams/SAC/2015.html                                      0.108108   
                             /teams/SAS/2015.html                                      0.069252   
                             /teams/TOR/2015.html                                     -0.517730   
                             /teams/UTA/2015.html                                      0.080702   
                             /teams/WAS/2015.html                                      0.382920   

                                                                        
                                                                        
                                                         PF       ORB   
                       index Team_id                                    
/leagues/NBA_2023.html 15    /teams/ATL/2023.html -0.860465  0.093333  \
                             /teams/BOS/2023.html -0.290323 -0.312500   
                             /teams/BRK/2023.html -0.032258 -0.562500   
                             /teams/CHI/2023.html -0.302326 -0.413333   


In [92]:
# Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_trf, y_trf, test_size=0.2, random_state=42)
X_train.shape,X_test.shape,y_train.shape,y_test.shape


((12950, 50), (3238, 50), (12950,), (3238,))

In [93]:
# Run models
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, roc_curve

def run_model(model, X_train, X_test, y_train, y_test, model_name, model_type, verbose=False):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:,1]
    y_pred_train = model.predict(X_train)
    y_pred_proba_train = model.predict_proba(X_train)[:,1]
    acc = accuracy_score(y_test, y_pred)
    acc_train = accuracy_score(y_train, y_pred_train)
    f1 = f1_score(y_test, y_pred)
    f1_train = f1_score(y_train, y_pred_train)
    prec = precision_score(y_test, y_pred)
    prec_train = precision_score(y_train, y_pred_train)
    rec = recall_score(y_test, y_pred)
    rec_train = recall_score(y_train, y_pred_train)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    roc_auc_train = roc_auc_score(y_train, y_pred_proba_train)
    conf_matrix = confusion_matrix(y_test, y_pred).reshape(-1)
    conf_matrix_train = confusion_matrix(y_train, y_pred_train).reshape(-1)
    if verbose:
        print(f'Accuracy: {acc:.4f} (train: {acc_train:.4f})')
        print(f'F1: {f1:.4f} (train: {f1_train:.4f})')
        print(f'Precision: {prec:.4f} (train: {prec_train:.4f})')
        print(f'Recall: {rec:.4f} (train: {rec_train:.4f})')
        print(f'ROC AUC: {roc_auc:.4f} (train: {roc_auc_train:.4f})')
        print(f'Confusion Matrix: {conf_matrix} (train: {conf_matrix_train}))')
    return {'model_name': model_name, 'model_type': model_type, 'acc': acc, 'acc_train': acc_train, 
            'f1': f1, 'f1_train': f1_train, 'prec': prec, 'prec_train': prec_train, 'rec': rec, 'rec_train': rec_train, 
            'roc_auc': roc_auc, 'roc_auc_train': roc_auc_train, 'conf_matrix': conf_matrix, 'conf_matrix_train': conf_matrix_train}


# Logistic Regression
model = LogisticRegressionCV(cv=3)
model_name = 'Logistic Regression'
model_type = 'classification'
logreg = run_model(model, X_train, X_test, y_train, y_test, model_name, model_type, verbose=True)
logreg



Accuracy: 0.6742 (train: 0.6610)
F1: 0.6761 (train: 0.6594)
Precision: 0.6792 (train: 0.6606)
Recall: 0.6730 (train: 0.6581)
ROC AUC: 0.7310 (train: 0.7151)
Confusion Matrix: [1082  520  535 1101] (train: [4311 2183 2207 4249]))


{'model_name': 'Logistic Regression',
 'model_type': 'classification',
 'acc': 0.6741815935762816,
 'acc_train': 0.661003861003861,
 'f1': 0.6760822843107154,
 'f1_train': 0.659373060211049,
 'prec': 0.6792103639728563,
 'prec_train': 0.6606032338308457,
 'rec': 0.6729828850855746,
 'rec_train': 0.6581474597273854,
 'roc_auc': 0.7310234914181235,
 'roc_auc_train': 0.7150795043294181,
 'conf_matrix': array([1082,  520,  535, 1101]),
 'conf_matrix_train': array([4311, 2183, 2207, 4249])}

In [219]:
import pandas as pd
from sklearn.linear_model import LinearRegression, RidgeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.datasets import make_classification, make_regression
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, accuracy_score, f1_score, precision_score, recall_score
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.svm import SVC, SVR


class ModelFinder:
    MODELS_CLASSIFICATION = {
        'RandomForestClassifier': RandomForestClassifier(),
        'LogisticRegression': LogisticRegression(),
        'RidgeClassifier': RidgeClassifier(),
        'SVC': SVC(),
        'MLPClassifier': MLPClassifier(),
    }
    MODELS_REGRESSION = {
        'RandomForestRegressor': RandomForestRegressor(),
        'Ridge': Ridge(),
        'SVR': SVR(),
        'LinearRegression': LinearRegression(),
        'MLPRegressor': MLPRegressor(),
    }
    MODEL_PARAMS_CLASSIFICATION = {
        'RandomForestClassifier': {'n_estimators': 100},
        'LogisticRegression': {'max_iter': 1000},
        'RidgeClassifier': {},
        'SVC': {},
        'MLPClassifier': {},
    }
    MODEL_PARAMS_REGRESSION = {
        'RandomForestRegressor': {'n_estimators': 100},
        'Ridge': {},
        'SVR': {},
        'LinearRegression': {},
        'MLPRegressor': {},
    }
    SCORES_CLASSIFICATION = ['accuracy','f1','precision','recall']
    SCORES_REGRESSION = ['mse','r2','mae','rmse']

    def __init__(self, models, model_params):
        self.models = models
        self.model_params = model_params
        self.predictions = None
        self.scores = None
        self.__init_models__()

    def __init_models__(self):
        for model_name, model in self.models.items():
            model.set_params(**self.model_params[model_name])
            self.models[model_name] = model

    def fit_predict(self, X, y, **kwargs):
        X_train, X_val, y_train, y_val = train_test_split(X, y, **kwargs)

        train_predictions = pd.DataFrame(index=X_train.index)
        val_predictions = pd.DataFrame(index=X_val.index)

        for model_name, model in self.models.items():
            fitted_model = model.fit(X_train, y_train)
            train_predictions[model_name] = fitted_model.predict(X_train)
            val_predictions[model_name] = fitted_model.predict(X_val)
        train_predictions.insert(0, 'y_true', y_train)
        val_predictions.insert(0, 'y_true', y_val)
        # Concatenating and adding labels
        predictions = pd.concat([train_predictions, val_predictions], keys=['train', 'val'], names=['type', 'index'])
        self.predictions = predictions
        return predictions
    
    def score(self, predictions, score_names):
        scores = {}
        for score_name in score_names:
            for model_name in self.models.keys():
                train_true = predictions.loc['train']['y_true']
                train_pred = predictions.loc['train'][model_name]
                val_true = predictions.loc['val']['y_true']
                val_pred = predictions.loc['val'][model_name]

                train_score = self._calculate_score(train_true, train_pred, score_name)
                val_score = self._calculate_score(val_true, val_pred, score_name)
                scores[(model_name, 'train', score_name)] = train_score
                scores[(model_name, 'val', score_name)] = val_score

        scores = pd.Series(scores)
        self.scores = scores
        return scores

    def _calculate_score(self, y_true, y_pred, score_name):
        # Classification metrics
        if score_name == 'accuracy':
            return accuracy_score(y_true, y_pred)
        elif score_name == 'f1':
            return f1_score(y_true, y_pred)
        elif score_name == 'precision':
            return precision_score(y_true, y_pred)
        elif score_name == 'recall':
            return recall_score(y_true, y_pred)
        
        # Regression metrics
        elif score_name == 'mse':
            return mean_squared_error(y_true, y_pred)
        elif score_name == 'r2':
            return r2_score(y_true, y_pred)
        elif score_name == 'mae':
            return mean_absolute_error(y_true, y_pred)
        elif score_name == 'rmse':
            return mean_squared_error(y_true, y_pred, squared=False)
        else:
            raise ValueError(f"Unknown score name: {score_name}")
        
    def get_params(self):
        params_dict = {}
        for model_name, model in self.models.items():
            params_dict[model_name] = (model.__class__.__name__, self.model_params[model_name])
        return params_dict

class ModelFinderCV(ModelFinder):
    def __init__(self, models, model_params):
        super().__init__(models, model_params)

    def fit_predict(self, X, y, cv=3, **kwargs):
        predictions = pd.DataFrame()
        kf_predictions_dict = {}
        kf = KFold(n_splits=cv,**kwargs)
        for i, (train_index, val_index) in enumerate(kf.split(X)):
            X_train, X_val = X.iloc[train_index], X.iloc[val_index]
            y_train, y_val = y.iloc[train_index], y.iloc[val_index]

            kf_train_predictions = pd.DataFrame(index=X_train.index)
            kf_val_predictions = pd.DataFrame(index=X_val.index)

            for model_name, model in self.models.items():
                fitted_model = model.fit(X_train, y_train)
                kf_train_predictions[model_name] = fitted_model.predict(X_train)
                kf_val_predictions[model_name] = fitted_model.predict(X_val)

            kf_train_predictions.insert(0, 'is_Val', 0)
            kf_val_predictions.insert(0, 'is_Val', 1)
            kf_predictions = pd.concat([kf_train_predictions, kf_val_predictions])
            kf_predictions_dict[f'fold_{i}'] = kf_predictions

        predictions = pd.concat(kf_predictions_dict.values(), keys=kf_predictions_dict.keys(),axis=1).reorder_levels([1,0],axis=1).sort_index(axis=1).sort_index(axis=0)
        predictions.insert(0, 'y_true', y)    
        self.predictions = predictions
        return predictions


    def score(self, predictions, score_names, avg_cv=True):
        scores = {}
        for model_name in self.models.keys():
            for cv_fold in predictions[model_name].columns:
                is_val_fold = predictions['is_Val'][cv_fold]
                train_index_fold = is_val_fold[is_val_fold == 0].index
                val_index_fold = is_val_fold[is_val_fold == 1].index
                
                train_true_fold = predictions.loc[train_index_fold]['y_true']
                train_pred_fold = predictions.loc[train_index_fold][model_name][cv_fold]
                val_true_fold = predictions.loc[val_index_fold]['y_true']
                val_pred_fold = predictions.loc[val_index_fold][model_name][cv_fold]

                for score_name in score_names:
                    train_score_fold = self._calculate_score(train_true_fold, train_pred_fold, score_name)
                    val_score_fold = self._calculate_score(val_true_fold, val_pred_fold, score_name)
                    scores[( score_name,'train', model_name, cv_fold, )] = train_score_fold
                    scores[(score_name, 'val', model_name, cv_fold, )] = val_score_fold
        scores = pd.Series(scores)
        if avg_cv:
            scores = scores.groupby(level=[0,1,2]).mean()

        self.scores = scores
        return scores
    

model_finder = ModelFinderCV(ModelFinder.MODELS_CLASSIFICATION, ModelFinder.MODEL_PARAMS_CLASSIFICATION)
X, y = make_classification(n_samples=1000, n_features=20, random_state=2)  # Example dataset
X = pd.DataFrame(X)
y = pd.Series(y)
predictions = model_finder.fit_predict(X, y, cv=3)
scores = model_finder.score(predictions, ModelFinder.SCORES_CLASSIFICATION)

In [211]:
# pd.MultiIndex.from_frame(predictions['is_Val'])
predictions
scores['f1']['val']

LogisticRegression        0.872691
MLP                       0.852605
RandomForestClassifier    0.895710
RidgeClassifier           0.870858
SVC                       0.868097
dtype: float64

In [220]:
model_finder.get_params()

{'RandomForestClassifier': ('RandomForestClassifier', {'n_estimators': 100}),
 'LogisticRegression': ('LogisticRegression', {'max_iter': 1000}),
 'RidgeClassifier': ('RidgeClassifier', {}),
 'SVC': ('SVC', {}),
 'MLP': ('MLPClassifier', {})}